In [3]:
import pandas as pd
import regex as re
import openpyxl
import numpy as np

In [2]:
def _get_time_row(df: pd.DataFrame) -> pd.Series:
    """Get the time row from the dataframe."""
    for row in df.iterrows():
        if re.match(r"^\d{1,2}:\d{1,2}-\d{1,2}:\d{1,2}$", str(row[1].iloc[1])):
            return row

In [4]:
def _get_daily_table(df: pd.DataFrame, class_pattern: str) -> pd.DataFrame:
    """
    Get the a simplified dataframe of the classes for a given class.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe to get the simplified time table from.
        It's a general time table on a single day for all classes.
    class_pattern : str
        The class to search for. E.g. 'EL 3'

    Returns
    -------
    pandas.DataFrame
        The simplified dataframe for only the given class.
    """
    df = df.copy()

    time_row = _get_time_row(df)
    new_cols = time_row[1].to_list()
    new_cols.pop(0)
    new_cols.insert(0, "Classroom")
    df.columns = new_cols

    df.set_index("Classroom", inplace=True)

    df = df.iloc[time_row[0]+1:]

    df = df.mask(~df.map(lambda x: bool(re.search(class_pattern, str(x)))))
    df = df.dropna(how='all')

    return df


def _get_all_daily_tables(filname: str, class_pattern: str) -> dict:
    """
    Get all the daily tables from an excel file.

    Parameters
    ----------
    filname : str
        The filename of the excel file to get the daily tables from.
    class_pattern : str
        The class to get the daily tables or. E.g. 'EL 3'

    Returns
    -------
    dict
        A dictionary of the daily tables for each class.
    """
    workbook = openpyxl.load_workbook(filname)
    dfs = {}
    for sheet in workbook.sheetnames:
        merged_cells = workbook[sheet].merged_cells.ranges
        for mc in merged_cells.copy():
            if mc.max_col - mc.min_col == 1:
                merged_value = workbook[sheet].cell(mc.min_row, mc.min_col).value
                workbook[sheet].unmerge_cells(mc.coord)
                workbook[sheet].cell(mc.min_row, mc.min_col).value = merged_value
                workbook[sheet].cell(mc.max_row, mc.max_col).value = merged_value

        data = workbook[sheet].values
        header = next(data)
        df = pd.DataFrame(data, columns=header)
        df = df.dropna(axis=1, how="all")

        dfs[sheet] = _get_daily_table(df, class_pattern)

    return dfs

In [5]:
def get_time_table(filname: str, class_pattern: str) -> pd.DataFrame:
    """
    Get the complete time table for a particular class for all days.

    Parameters
    ----------
    filname : str
        The filename of the excel file. This file contains every class with the days as the sheet names. 
    class_pattern : str
        The class to get the complete time table for. E.g. 'EL 3'

    Returns
    -------
    pandas.DataFrame
        The complete time table for the given class.
    """
    daily_tables = _get_all_daily_tables(filname, class_pattern)

    
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    for key, value in daily_tables.items():
        if key.title() in days:
            columns = value.columns
            break
    else:
        raise ValueError(f"No sheet found for any of the days: {days}")

    final_df = pd.DataFrame(
        columns=columns,
        index=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"],
    )

    for day, table in daily_tables.items():
        for period, classes in table.items():
            available_classes = classes.dropna()
            if available_classes.any():
                classrooms = classes[classes.notna()].index
                available_classes = [re.sub(r'\s+', ' ', c.strip()) for c in available_classes.values]
                available_classes = [f"{c} ({classrooms[i]})" for i, c in enumerate(available_classes)]
                available_classes = '\n'.join(available_classes)
                final_df.loc[day, period] = available_classes

    return final_df



In [ ]:
df = get_time_table("Draft_4.xlsx", "EL 3")
writer = pd.ExcelWriter(filename, engine="xlsxwriter")
df.to_excel(writer, index=False, sheet_name="Sheet1")

workbook = writer.book
worksheet = writer.sheets["Sheet1"]
worksheet.set_zoom(75)

In [2]:
class Bisection(object):

    def __init__(self, prob_func:str, interval:str):
        self.prob_func = prob_func
        self.interval = interval.split(',')
        self._limit1_eval = None
        self._limit2_eval = None
        self.middle_limit = None
        self.iteration = 0
    
    def findIndependentVar(self):
        return [self.prob_func[i+1] for i in range(len(self.prob_func)) if self.prob_func[i] == '('][0]
    def satisfyAssumptions(self, interval):
        """
        for a problem to be solved using bisection method
        the function must be continous
        the product of the evaluation of the limits must be less than zero
        difference between xi and xu is a very small value
        """
        infinity = float('inf')
        self._limit1_eval = self.evaluate(interval[0])
        self._limit2_eval = self.evaluate(interval[1])
        print('limit 1: ',self._limit1_eval, 'limit 2: ',self._limit2_eval, 'product=', self._limit1_eval * self._limit2_eval)

        return all([self._limit1_eval != infinity, self._limit2_eval != infinity, self._limit1_eval * self._limit2_eval < 0])
    
    
    def evaluate(self,interval_elment:str):
       return eval(self.prob_func.replace(self.findIndependentVar(), interval_elment).split('=')[1])

    def bisect(self):
        self.iteration += 1
        if self.iteration == 10:
            print('iter @ 10:---------------------------------------- ', self.middle_limit)
        self.middle_limit = (float(self.interval[0]) + float(self.interval[1]))/ 2
        print(self.middle_limit)
        if self.evaluate(str(self.middle_limit)) == 0:
            """return the root"""
            return print('the correct estimated root is: ', self.middle_limit,
                         'at the end of iteration: ', self.iteration)
        else: 
            if self.satisfyAssumptions([self.interval[0]] + [str(self.middle_limit)]):
                print('entered first interval')
                self.interval = [self.interval[0]] + [str(self.middle_limit)]
                self.bisect()
            elif self.satisfyAssumptions([self.interval[1]] + [str(self.middle_limit)]):
                print('entered second interval')
                self.interval = [self.interval[1]] + [str(self.middle_limit)]
                self.bisect()
            else:
                return "given function doesn’t follow one of assumptions."


first = Bisection('f(x) = x**3 - 0.187*x**2 + 4.993 * 10**-4', '0,0.11')
first.bisect()

0.055
limit 1:  0.0004993 limit 2:  0.00010000000000000005 product= 4.9930000000000024e-08
limit 1:  -0.0004323999999999999 limit 2:  0.00010000000000000005 product= -4.324000000000001e-08
entered second interval
0.0825
limit 1:  -0.0004323999999999999 limit 2:  -0.000211953125 product= 9.164853124999998e-08
limit 1:  0.00010000000000000005 limit 2:  -0.000211953125 product= -2.119531250000001e-08
entered second interval
0.06875
limit 1:  0.00010000000000000005 limit 2:  -5.961601562499996e-05 product= -5.961601562499999e-09
entered first interval
0.061875
limit 1:  0.00010000000000000005 limit 2:  2.0256982421875097e-05 product= 2.0256982421875107e-09
limit 1:  -5.961601562499996e-05 limit 2:  2.0256982421875097e-05 product= -1.2076405805778552e-09
entered second interval
0.0653125
limit 1:  -5.961601562499996e-05 limit 2:  -1.978512573242168e-05 product= 1.1795103648066398e-09
limit 1:  2.0256982421875097e-05 limit 2:  -1.978512573242168e-05 product= -4.007869441762546e-10
entered se